In [ ]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification
from transformers import AutoTokenizer, AutoModel

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 48.9 MB/s 
     |████████████████████████████████| 163 kB 40.4 MB/s 


In [ ]:
import pandas as pd
df = pd.read_json('https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/train.json')

In [ ]:
dfsetfinal = pd.DataFrame(columns=['DocNo', 'Text', 'Labels'])

In [ ]:
rows_list=[]
for i in range(len(df)):
  for rec in df["annotations"][i]:
    for line in rec["result"]:
      rows_list.append({"DocNo":i,"Text":line["value"]["text"],"Labels":line["value"]["labels"][0]})
      if(len(line["value"]["labels"]))>=2:
        print("Yes")
dfsetfinal = pd.DataFrame(rows_list)

In [ ]:
dfsetfinal

,DocNo,Text,Labels
0,0,"IN THE HIGH COURT OF KARNATAKA,\n ...",PREAMBLE
1,0,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,PREAMBLE
2,0,This Criminal Appeal is filed under Section 37...,PREAMBLE
3,0,\n\n This appeal coming on for hearing t...,PREAMBLE
4,0,\n Heard the learned Counsel for the app...,NONE
...,...,...,...
28981,246,\n So Section 132 of the Evidence Act sufficie...,RATIO
28982,246,"\n For the reasons aforesaid, the appeal is al...",RPC
28983,246,"The judgment and order dated April 27, 1987 pa...",RPC
28984,246,\n R.S.S.,NONE


In [ ]:
#temp_df=dfsetfinal.query('DocNo<3')

In [ ]:
dfsetfinal

,DocNo,Text,Labels
0,0,"IN THE HIGH COURT OF KARNATAKA,\n ...",PREAMBLE
1,0,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,PREAMBLE
2,0,This Criminal Appeal is filed under Section 37...,PREAMBLE
3,0,\n\n This appeal coming on for hearing t...,PREAMBLE
4,0,\n Heard the learned Counsel for the app...,NONE
...,...,...,...
28981,246,\n So Section 132 of the Evidence Act sufficie...,RATIO
28982,246,"\n For the reasons aforesaid, the appeal is al...",RPC
28983,246,"The judgment and order dated April 27, 1987 pa...",RPC
28984,246,\n R.S.S.,NONE


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

ModuleNotFoundError: ignored

In [ ]:
dfsetfinal.groupby('Labels').describe()
#temp_df.groupby('Labels').describe()

DocNo                                                   \
                  count        mean        std  min    25%    50%    75%   
Labels                                                                     
ANALYSIS        10695.0  121.289668  71.765943  0.0  62.00  122.0  183.0   
ARG_PETITIONER   1315.0  146.024335  67.677526  0.0  95.00  149.0  213.0   
ARG_RESPONDENT    698.0  126.693410  70.550880  0.0  62.00  121.0  182.0   
FAC              5744.0  122.981720  75.609740  0.0  46.00  126.0  192.0   
ISSUE             367.0  127.378747  70.887241  1.0  67.00  136.0  183.0   
NONE             1423.0  121.867182  71.128308  0.0  52.00  125.0  179.0   
PREAMBLE         4167.0  125.306935  72.077698  0.0  64.00  128.0  188.0   
PRE_NOT_RELIED    158.0  112.240506  61.548120  3.0  65.00  108.0  166.0   
PRE_RELIED       1431.0  117.864430  79.692853  0.0  50.00  109.0  202.0   
RATIO             674.0  120.770030  71.863906  0.0  54.25  125.0  179.0   
RLC               752.0  103.873670  69.951334  0.0  42.00   93.0  171.0   
RPC              1081.0  125.368178  73.066975  0.0  66.00  124.0  188.0   
STA               481.0  115.683992  74.274743  3.0  45.00   94.0  181.0   

                       
                  max  
Labels                 
ANALYSIS        246.0  
ARG_PETITIONER  245.0  
ARG_RESPONDENT  246.0  
FAC             246.0  
ISSUE           245.0  
NONE            246.0  
PREAMBLE        246.0  
PRE_NOT_RELIED  232.0  
PRE_RELIED      246.0  
RATIO           246.0  
RLC             246.0  
RPC             246.0  
STA             246.0

In [ ]:
possible_labels = dfsetfinal.Labels.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict
#possible_labels = temp.Labels.unique()

#label_dict = {}
#for index, possible_label in enumerate(possible_labels):
#    label_dict[possible_label] = index
#label_dict

{'PREAMBLE': 0,
 'NONE': 1,
 'FAC': 2,
 'ARG_RESPONDENT': 3,
 'RLC': 4,
 'ARG_PETITIONER': 5,
 'ANALYSIS': 6,
 'PRE_RELIED': 7,
 'RATIO': 8,
 'RPC': 9,
 'ISSUE': 10,
 'STA': 11,
 'PRE_NOT_RELIED': 12}

In [ ]:
dfsetfinal['LabelsValue']=dfsetfinal['Labels'].apply(lambda x: label_dict[x])
#temp_df

In [ ]:
dfsetfinal

,DocNo,Text,Labels,LabelsValue
0,0,"IN THE HIGH COURT OF KARNATAKA,\n ...",PREAMBLE,0
1,0,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,PREAMBLE,0
2,0,This Criminal Appeal is filed under Section 37...,PREAMBLE,0
3,0,\n\n This appeal coming on for hearing t...,PREAMBLE,0
4,0,\n Heard the learned Counsel for the app...,NONE,1
...,...,...,...,...
28981,246,\n So Section 132 of the Evidence Act sufficie...,RATIO,8
28982,246,"\n For the reasons aforesaid, the appeal is al...",RPC,9
28983,246,"The judgment and order dated April 27, 1987 pa...",RPC,9
28984,246,\n R.S.S.,NONE,1


In [ ]:
#from sklearn.model_selection import train_test_split
X_train = dfsetfinal.filter(['Text'],axis=1)
y_train = dfsetfinal.filter(['LabelsValue'], axis =1)
#X_train, X_val, y_train, y_val = train_test_split(dfsetfinal['Text'], 
#                                                  dfsetfinal['LabelsValue'], 
#                                                  test_size=0, 
#                                                  random_state=42, 
#                                                  stratify=dfsetfinal['LabelsValue'])

In [ ]:
dfsetfinal['Text'][0]

'      IN THE HIGH COURT OF KARNATAKA,\n          CIRCUIT BENCH AT GULBARGA\n\nDATED THIS THE 22ND DAY OF FEBRUARY, 2013'

In [ ]:
X_train_vals=[]
for i in X_train:
    X_train_vals.append(i)

In [ ]:
#X_val_vals=[]
#for i in X_val:
#    X_val_vals.append(i)

In [ ]:
i=X_train.index

In [ ]:
print(i)

RangeIndex(start=0, stop=28986, step=1)


In [ ]:
#dfsetfinal.loc[i, 'data_type'] = 'train'

In [ ]:
dfsetfinal

,DocNo,Text,Labels,LabelsValue
0,0,"IN THE HIGH COURT OF KARNATAKA,\n ...",PREAMBLE,0
1,0,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,PREAMBLE,0
2,0,This Criminal Appeal is filed under Section 37...,PREAMBLE,0
3,0,\n\n This appeal coming on for hearing t...,PREAMBLE,0
4,0,\n Heard the learned Counsel for the app...,NONE,1
...,...,...,...,...
28981,246,\n So Section 132 of the Evidence Act sufficie...,RATIO,8
28982,246,"\n For the reasons aforesaid, the appeal is al...",RPC,9
28983,246,"The judgment and order dated April 27, 1987 pa...",RPC,9
28984,246,\n R.S.S.,NONE,1


In [ ]:
#j=X_val.index

In [ ]:
#dfsetfinal.loc[j, 'data_type'] = 'val'

In [ ]:
dfsetfinal

,DocNo,Text,Labels,LabelsValue
0,0,"IN THE HIGH COURT OF KARNATAKA,\n ...",PREAMBLE,0
1,0,\n\n BEFORE\n\nTHE HON'BLE MR.JUSTICE ANA...,PREAMBLE,0
2,0,This Criminal Appeal is filed under Section 37...,PREAMBLE,0
3,0,\n\n This appeal coming on for hearing t...,PREAMBLE,0
4,0,\n Heard the learned Counsel for the app...,NONE,1
...,...,...,...,...
28981,246,\n So Section 132 of the Evidence Act sufficie...,RATIO,8
28982,246,"\n For the reasons aforesaid, the appeal is al...",RPC,9
28983,246,"The judgment and order dated April 27, 1987 pa...",RPC,9
28984,246,\n R.S.S.,NONE,1


In [ ]:
# dfsetfinal.groupby(['Labels', 'LabelsValue', 'data_type']).count()
dfsetfinal.groupby(['Labels', 'LabelsValue']).count()

,,DocNo,Text
Labels,LabelsValue,,
ANALYSIS,6,10695,10695
ARG_PETITIONER,5,1315,1315
ARG_RESPONDENT,3,698,698
FAC,2,5744,5744
ISSUE,10,367,367
NONE,1,1423,1423
PREAMBLE,0,4167,4167
PRE_NOT_RELIED,12,158,158
PRE_RELIED,7,1431,1431


In [ ]:
print(dfsetfinal['LabelsValue'])

0        0
1        0
2        0
3        0
4        1
        ..
28981    8
28982    9
28983    9
28984    1
28985    9
Name: LabelsValue, Length: 28986, dtype: int64


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("law-ai/InLegalBERT")

Downloading:   0%|          | 0.00/516 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/222k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
dfsetfinal.Text.values

array(['      IN THE HIGH COURT OF KARNATAKA,\n          CIRCUIT BENCH AT GULBARGA\n\nDATED THIS THE 22ND DAY OF FEBRUARY, 2013',
       "\n\n      BEFORE\n\nTHE HON'BLE MR.JUSTICE ANAND BYRAREDDY\n\n      CRIMINAL APPEAL NO.3532 OF 2012\n\nBETWEEN:                                            R\n\nBabu S/o Siddappa,                         .. APPELLANT\nAge: 30 Years, Occ: Household,\nR/o: Sunthan Village,\nTaluk Chincholi,\nDistrict Gulbarga.\n\n(By Shri Ishwar Raj S.Chowdapur, Advocate)\n\nAND:\n\nThe State of Karnataka                     .. RESPONDENT\n(Through Ratkal Police Station)\nRepresented by Additional State\nPublic Prosecutor, Circuit Bench,\nGulbarga.\n\n(By Shri S.S.Aspalli, Government Pleader)\n\n       ",
       'This Criminal Appeal is filed under Section 374(2) of the\nCode of Criminal Procedure, 1973 by the advocate for the\nappellant praying to set aside the order of conviction and\n\nsentence in S.C.No.232/2008 on the file of the II Additional\nSessions Judge, Gulb

In [ ]:
encoded_data_train = tokenizer.batch_encode_plus(
    list(dfsetfinal.Text.values), 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

#encoded_data_val = tokenizer.batch_encode_plus(
#    temp_df[temp_df.data_type=='val'].Text.values, 
#    add_special_tokens=True, 
#    return_attention_mask=True, 
#    pad_to_max_length=True, 
#    max_length=256, 
#    return_tensors='pt'
#)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
#dfsetfinal[dfsetfinal.data_type=='train'].LabelsValue.values
dfsetfinal.LabelsValue.values

array([0, 0, 0, ..., 9, 1, 9])

In [ ]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(dfsetfinal.LabelsValue.values)

#input_ids_val = encoded_data_val['input_ids']
#attention_masks_val = encoded_data_val['attention_mask']
#labels_val = torch.tensor(temp_df[temp_df.data_type=='val'].LabelsValue.values)

In [ ]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
#dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_train)

28986

In [ ]:
model = BertForSequenceClassification.from_pretrained("law-ai/InLegalBERT",
                                  num_labels=len(label_dict),
                                  output_attentions=False,
                                  output_hidden_states=False)

Downloading:   0%|          | 0.00/671 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/534M [00:00<?, ?B/s]

Some weights of the model checkpoint at law-ai/InLegalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initial

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

#dataloader_validation = DataLoader(dataset_val, 
#                                   sampler=SequentialSampler(dataset_val), 
#                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
epochs = 6

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [ ]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/sample_data/InLegalBERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    #val_loss, predictions, true_vals = evaluate(dataloader_validation)
    #val_f1 = f1_score_func(predictions, true_vals)
    #tqdm.write(f'Validation loss: {val_loss}')
    #tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/6 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/9662 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.7040475551667355


Epoch 2:   0%|          | 0/9662 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.5793823998074992


...


Epoch 4
Training loss: 0.43028138299383517


Epoch 5:   0%|          | 0/9662 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.42489183541964837


Epoch 6:   0%|          | 0/9662 [00:00<?, ?it/s]


Epoch 6
Training loss: 0.431303208677694


In [ ]:
model.load_state_dict(torch.load('/content/sample_data/InLegalBERT_epoch_5.model', map_location=torch.device('cpu')))

RuntimeError: ignored

In [ ]:
df_val = pd.read_json('https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Data/dev.json')

In [ ]:
dfvalsetfinal = pd.DataFrame(columns=['DocNo', 'Text', 'Labels'])

In [ ]:
rows_vallist=[]
for i in range(len(df_val)):
  for rec in df_val["annotations"][i]:
    for line in rec["result"]:
      rows_vallist.append({"DocNo":i,"Text":line["value"]["text"],"Labels":line["value"]["labels"][0]})
      if(len(line["value"]["labels"]))>=2:
        print("Yes")
dfvalsetfinal = pd.DataFrame(rows_vallist)

In [ ]:
print(dfvalsetfinal)

      DocNo                                               Text    Labels
0         0  PETITIONER:\nTHE COMMISSIONER OF INCOME-TAXNEW...  PREAMBLE
1         0  \n\nDATE OF JUDGMENT:\n05/05/1961\n\nBENCH:\nD...  PREAMBLE
2         0  \nBENCH:\nDAS, S.K.\nHIDAYATULLAH, M.\nSHAH, J...  PREAMBLE
3         0  It\nentered into transactions in the nature of...  PREAMBLE
4         0  The assessee claimed deduction of these\nlosse...  PREAMBLE
...     ...                                                ...       ...
2885     29                         The petitions are allowed.       RPC
2886     29  The impugned orders are set aside with directi...       RPC
2887     29  \n The respondent having challenged the judgme...       RPC
2888     29  Therefore, having regard to the law laid down ...       RPC
2889     29                                   \n Sd/- JUDGE nv      NONE

[2890 rows x 3 columns]


In [ ]:
dfvalsetfinal['LabelsValue']=dfvalsetfinal['Labels'].apply(lambda x: label_dict[x])

In [ ]:
dfvalsetfinal

,DocNo,Text,Labels,LabelsValue
0,0,PETITIONER:\nTHE COMMISSIONER OF INCOME-TAXNEW...,PREAMBLE,0
1,0,\n\nDATE OF JUDGMENT:\n05/05/1961\n\nBENCH:\nD...,PREAMBLE,0
2,0,"\nBENCH:\nDAS, S.K.\nHIDAYATULLAH, M.\nSHAH, J...",PREAMBLE,0
3,0,It\nentered into transactions in the nature of...,PREAMBLE,0
4,0,The assessee claimed deduction of these\nlosse...,PREAMBLE,0
...,...,...,...,...
2885,29,The petitions are allowed.,RPC,9
2886,29,The impugned orders are set aside with directi...,RPC,9
2887,29,\n The respondent having challenged the judgme...,RPC,9
2888,29,"Therefore, having regard to the law laid down ...",RPC,9


In [ ]:
encoded_data_val = tokenizer.batch_encode_plus(
    dfvalsetfinal.Text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

C:\Users\Ramanan\AppData\Roaming\Python\Python39\site-packages\transformers\tokenization_utils_base.py:2184: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(dfvalsetfinal.LabelsValue.values)

In [ ]:
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [ ]:
len(dataset_val)

2890

In [ ]:
dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
model.load_state_dict(torch.load('inlegalbertrr/LegalRR_finetuned_BERT_epoch_1.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [ ]:
accuracy_per_class(predictions, true_vals)

Class: PREAMBLE
Accuracy: 323/509

Class: NONE
Accuracy: 167/192

Class: FAC
Accuracy: 431/581

Class: ARG_RESPONDENT
Accuracy: 0/38

Class: RLC
Accuracy: 25/116

Class: ARG_PETITIONER
Accuracy: 20/70

Class: ANALYSIS
Accuracy: 786/986

Class: PRE_RELIED
Accuracy: 57/142

Class: RATIO
Accuracy: 6/72

Class: RPC
Accuracy: 77/92

Class: ISSUE
Accuracy: 35/51

Class: STA
Accuracy: 19/29

Class: PRE_NOT_RELIED
Accuracy: 0/12

